In [4]:
%pip install torch torchvision --upgrade
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/109.3 MB ? eta -:--:--
   ---------------------------------------- 1.0/109.3 MB 7.1 MB/s eta 0:00:16
   - -------------------------------------- 4.7/109.3 MB 13.6 MB/s eta 0:00:08
   -- ------------------------------------- 6.6/109.3 MB 11.8 MB/s eta 0:00:09
   -- ------------------------------------- 7.3/109.3 MB 11.3 MB/s eta 0:00:10
   ---- ----------------------------------- 11.5/109.3 MB 11.8 MB/s eta 0:00:09
   ----- ---------------------------------- 15.7/109.3 MB 12.1 MB/s eta 0:00:08
   ------ --------------------------------- 18.6/109.3 MB 12.2 MB/s eta 0:00:08
   ------- -------------------------------- 20.7/109.3 MB 12.0 MB/s eta 0:00:08
   -------- ------------------------------- 23.3/109.3 MB 12.1 MB/s eta 0:00:08
   -------- ------------------------------- 24.4/109.3 MB 11.7 MB/s eta 0:00:08
   -------- ------------------------------- 24.4/109.3 MB 11.7 MB/s eta 

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.9.0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


ImportError: cannot import name 'profiler_allow_cudagraph_cupti_lazy_reinit_cuda12' from 'torch._utils_internal' (c:\Users\Ekaansh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils_internal.py)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])  
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])


In [ ]:
train_data = datasets.ImageFolder(r"D:\datasets\chest_xray\chest_xray\train", transform=train_transform)
val_data   = datasets.ImageFolder(r"D:\datasets\chest_xray\chest_xray\val", transform=val_test_transform)
test_data  = datasets.ImageFolder(r"D:\datasets\chest_xray\chest_xray\test", transform=val_test_transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
y_train = [label for _, label in train_data.samples]
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
print("Class Weights:", class_weights)

Class Weights: tensor([1.9448, 0.6730], device='cuda:0')


In [ ]:
model = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)


num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 2)  
)

In [ ]:
for name, param in model.named_parameters():
    param.requires_grad = False
    if "denseblock3" in name or "denseblock4" in name or "classifier" in name:
        param.requires_grad = True

model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=10)


In [ ]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_acc = 0
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total * 100
        val_acc = evaluate_model(model, val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_pneumonia_densenet121.pt")


In [ ]:
def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total * 100

In [ ]:

train_model(model, train_loader, val_loader, epochs=10)

Epoch 1/10, Loss: 0.2653, Train Acc: 91.55%, Val Acc: 87.50%
Epoch 2/10, Loss: 0.1324, Train Acc: 95.28%, Val Acc: 93.75%
Epoch 3/10, Loss: 0.1325, Train Acc: 94.94%, Val Acc: 100.00%
Epoch 4/10, Loss: 0.1026, Train Acc: 96.20%, Val Acc: 93.75%
Epoch 5/10, Loss: 0.0858, Train Acc: 97.24%, Val Acc: 100.00%
Epoch 6/10, Loss: 0.0721, Train Acc: 97.47%, Val Acc: 100.00%
Epoch 7/10, Loss: 0.0663, Train Acc: 97.60%, Val Acc: 81.25%
Epoch 8/10, Loss: 0.0496, Train Acc: 97.85%, Val Acc: 100.00%
Epoch 9/10, Loss: 0.0447, Train Acc: 98.43%, Val Acc: 93.75%
Epoch 10/10, Loss: 0.0399, Train Acc: 98.60%, Val Acc: 87.50%


In [ ]:
model.load_state_dict(torch.load("best_pneumonia_densenet121.pt"))
model.eval()

C:\Users\Ekaansh\AppData\Local\Temp\ipykernel_18684\1610110734.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_pneumonia_densenet1

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=["PNEUMONIA", "NORMAL"]))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


Classification Report:
               precision    recall  f1-score   support

   PNEUMONIA       0.98      0.86      0.91       234
      NORMAL       0.92      0.99      0.95       390

    accuracy                           0.94       624
   macro avg       0.95      0.92      0.93       624
weighted avg       0.94      0.94      0.94       624


Confusion Matrix:
 [[201  33]
 [  5 385]]
